In [ ]:
import json
import random
import signal
import string
import time
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import paho.mqtt.client as mqtt

normalBinMax = 46
sanBinMax = 40
toiletpaperMax = 10
database_settings = { "username": "admin",
                     "password": "abhyuday98",
                     "ip_address":"iotg5.ccspnys1uzca.us-east-1.rds.amazonaws.com",
                     "port": "3306",
                     "database_name": "iotg5"
                            }
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(database_settings['username'],
                                                                              database_settings['password'],
                                                                              database_settings['ip_address'],
                                                                              database_settings['port'],
                                                                              database_settings['database_name']
                                                                              ))


KPC_HOST = "mqtt.cloud.kaaiot.com"  # Kaa Cloud plain MQTT host
KPC_PORT = 1883  # Kaa Cloud plain MQTT port

ENDPOINT_TOKEN = "DfxM96LkQT" # Paste endpoint token
APPLICATION_VERSION = "bu29218547tsntf2sup0-v1"  # Paste application version


class KaaClient:

    def __init__(self, client):
        self.client = client
        self.metadata_update_topic = f'kp1/{APPLICATION_VERSION}/epmx/{ENDPOINT_TOKEN}/update/keys'
        self.data_collection_topic = f'kp1/{APPLICATION_VERSION}/dcx/{ENDPOINT_TOKEN}/json'

    def connect_to_server(self):
        print(f'Connecting to Kaa server at {KPC_HOST}:{KPC_PORT} using application version {APPLICATION_VERSION} and endpoint token {ENDPOINT_TOKEN}')
        self.client.connect(KPC_HOST, KPC_PORT, 60)
        print('Successfully connected')

    def disconnect_from_server(self):
        print(f'Disconnecting from Kaa server at {KPC_HOST}:{KPC_PORT}...')
        self.client.loop_stop()
        self.client.disconnect()
        print('Successfully disconnected')

    def compose_metadata(self):
        return json.dumps([
            {
                "model": "BME/BMP 280",
                "fwVersion": "v0.0.1",
                "customer": "Andrew",
                "latitude": 40.71427,
                "longitude": -74.00597,
            }
        ])
    

        
    
    def compose_data_sample(self):
        normalBinMax = 46
        sanBinMax = 30
        toiletpaperMax = 9

        def inbetweenRange(x, lower, upper):
        #     if(x<=lower):
        #         return float(lower)
        #     if(x>=upper):
        #         return float(upper)
            return float(x)

        def toiletpaperLevel(x):
            print('toiletpaperLevel',x)
            return ((inbetweenRange(x, 0, toiletpaperMax)*100)//toiletpaperMax)

        def sanBinLevel(x):
            print('sanBinLevel',x)

            return ((sanBinMax-(inbetweenRange(x, 0, sanBinMax)))*100)//sanBinMax

        def normalBinLevel(x):
            print('normalBinLevel',x)

            return ((normalBinMax-(inbetweenRange(x, 0, normalBinMax)))*100)//normalBinMax

        litterbin1 = 50
        litterbin2 = 50
        toiletpaper1 = 50
        toiletpaper2 = 50
        with engine.connect() as con:
            statement = text("select (SELECT `distance(cm)` from litterbin1 order by datetime desc limit 1) as litterbin1, (SELECT `distance(cm)` from litterbin2 order by datetime desc limit 1) as litterbin2,(SELECT `distance(cm)` from toiletpaper1 order by datetime desc limit 1) as toiletpaper1, (SELECT `distance(cm)` from toiletpaper2 order by datetime desc limit 1) as toiletpaper2;")
            result = con.execute(statement)
            
            for row in result:
                litterbin1 = normalBinLevel(row['litterbin1'])
                litterbin2 = sanBinLevel(row['litterbin2'])
                toiletpaper1 = 100 - toiletpaperLevel(row['toiletpaper1'])
                toiletpaper2 = 100 - toiletpaperLevel(row['toiletpaper2'])
                if(toiletpaper1<0):
                    toiletpaper1 = 0
                if(toiletpaper2<0):
                    toiletpaper2 = 0
#                 print('toiletpaper1:', 100-toiletpaper1,'toiletpaper2:', 100-toiletpaper2)
        return json.dumps([
            {
              "timestamp": int(round(time.time() * 1000)),
              "litterbin1": litterbin1,
              "litterbin2": litterbin2,
              "toiletpaper1": toiletpaper1,
              "toiletpaper2": toiletpaper2

            }
        ])


def on_message(client, userdata, message):
    print(f'<-- Received message on topic "{message.topic}":\n{str(message.payload.decode("utf-8"))}')


def main():
    # Initiate server connection
    client = mqtt.Client(client_id=''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(6)))

    kaa_client = KaaClient(client)
    kaa_client.connect_to_server()
    client.on_message = on_message

    # Start the loop
    client.loop_start()

    # Send data samples in loop
    listener = SignalListener()

    payload = kaa_client.compose_metadata()
    result = kaa_client.client.publish(topic=kaa_client.metadata_update_topic, payload=payload)
    print(f'--> Sent message on topic "{kaa_client.metadata_update_topic}":\n{payload}')
    
    while listener.keepRunning:
        payload = kaa_client.compose_data_sample()
        result = kaa_client.client.publish(topic=kaa_client.data_collection_topic, payload=payload)
        if result.rc != 0:
            print('Server connection lost, attempting to reconnect')
            kaa_client.connect_to_server()
        else:
            print(f'--> Sent message on topic "{kaa_client.data_collection_topic}":\n{payload}')
        time.sleep(3)

    kaa_client.disconnect_from_server()


class SignalListener:
    keepRunning = True

    def __init__(self):
        signal.signal(signal.SIGINT, self.stop)
        signal.signal(signal.SIGTERM, self.stop)

    def stop(self, signum, frame):
        print('Shutting down...')
        self.keepRunning = False


if __name__ == '__main__':
    main()


Connecting to Kaa server at mqtt.cloud.kaaiot.com:1883 using application version bu29218547tsntf2sup0-v1 and endpoint token DfxM96LkQT
Successfully connected
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/epmx/DfxM96LkQT/update/keys":
[{"model": "BME/BMP 280", "fwVersion": "v0.0.1", "customer": "Andrew", "latitude": 40.71427, "longitude": -74.00597}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512667308, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
<-- Received message on topic "kp1/bu29218547tsntf2sup0-v1/epmx/DfxM96LkQT/update/keys/error":
{"statusCode":400,"reasonPhrase":"Payload is not valid JSON object"}
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512670317, "litterbin1": 89.0, "litterbin2": 13.

normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512757590, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512760599, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512763606, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512766615, "litterbin1": 89.0, "litterbin2": 13.0, "t

normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512853875, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512856884, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512859893, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512862902, "litterbin1": 89.0, "litterbin2": 13.0, "t

normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512950163, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512953173, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512956182, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603512959191, "litterbin1": 89.0, "litterbin2": 13.0, "t

normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603513046447, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603513049457, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603513052467, "litterbin1": 89.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 67.0}]
normalBinLevel 5
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 3
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603513055476, "litterbin1": 89.0, "litterbin2": 13.0, "t

...

normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515068918, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515071928, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515074938, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515077947, "litterbin1": 93.0, "litterbin2": 13.0, "t

normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515165332, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515168342, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515171351, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515174361, "litterbin1": 93.0, "litterbin2": 13.0, "t

normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515261642, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515264651, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515267661, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515270669, "litterbin1": 93.0, "litterbin2": 13.0, "t

normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515357959, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515360968, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515363977, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515366995, "litterbin1": 93.0, "litterbin2": 13.0, "t

normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515454285, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515457297, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515460306, "litterbin1": 93.0, "litterbin2": 13.0, "toiletpaper1": 0, "toiletpaper2": 56.0}]
normalBinLevel 3
sanBinLevel 26
toiletpaperLevel 10
toiletpaperLevel 4
--> Sent message on topic "kp1/bu29218547tsntf2sup0-v1/dcx/DfxM96LkQT/json":
[{"timestamp": 1603515463316, "litterbin1": 93.0, "litterbin2": 13.0, "t